In [1]:
import tensorflow as tf 
import os
import pickle
import numpy as np
    
CIFAR_DIR = "../cifar-10-python/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


D:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename,'rb') as f:
        data = pickle.load(f, encoding='iso-8859-1')
        return data['data'],data['labels']
#数据处理
class CifarData:
    def __init__(self,filenames,need_shuffle): #训练集需要shuffl,训练完一遍后shuffl一下，使数据更加散乱。测试集不需要shuffl
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            for item,label in zip(data,labels):
                if label in[0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    def next_batch(self,batch_size):
        """return batch_size example as a abatch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels
        
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,True)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


In [3]:
x = tf.placeholder(tf.float32,[None,3072])     #3072为图片维度
y = tf.placeholder(tf.int64,[None])            #y的shape为[None]

w =  tf.get_variable('w',[x.get_shape()[-1],1], # 56165165
                     initializer=tf.random_normal_initializer(0,1)) #对w做初始化
b =  tf.get_variable('b',[1],
                     initializer=tf.constant_initializer(0.0))
                     
y_ = tf.matmul(x,w) + b  #此时y为内积值  ,y__shape = [None,3072] * [3072,1] = [None,1]
p_y_1 = tf.nn.sigmoid(y_)  #把y值变为一个概率值

#为了将y与p_y_1做差，所以要将y进行reshape
y_reshaped = tf.reshape(y,(-1,1))  #y_shape = [None,1]

#因为p_y_1是float32,y_reshape是int64,类型不一样，所以计算会出问题，需要对y_reshape做一个变换
y_reshaped_float = tf.cast(y_reshaped,tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1)) 
                     
#准确率
#bool
predict = p_y_1 > 0.5
#[1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.cast(predict,tf.int64),y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))
                     
#定义一个梯度下降的方法
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

# run 100k: 50.5%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f'
                  % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

[Train] Step: 500, loss: 0.45000, acc: 0.55000
[Train] Step: 1000, loss: 0.35000, acc: 0.65000
[Train] Step: 1500, loss: 0.40000, acc: 0.60000
[Train] Step: 2000, loss: 0.40000, acc: 0.60000
[Train] Step: 2500, loss: 0.55000, acc: 0.45000
[Train] Step: 3000, loss: 0.60000, acc: 0.40000
[Train] Step: 3500, loss: 0.15000, acc: 0.85000
[Train] Step: 4000, loss: 0.60000, acc: 0.40000
[Train] Step: 4500, loss: 0.50000, acc: 0.50000
[Train] Step: 5000, loss: 0.50000, acc: 0.50000
(2000, 3072)
(2000,)
[Test ] Step: 5000, acc: 0.50050
[Train] Step: 5500, loss: 0.40000, acc: 0.60000
[Train] Step: 6000, loss: 0.45000, acc: 0.55000
[Train] Step: 6500, loss: 0.65000, acc: 0.35000
[Train] Step: 7000, loss: 0.45000, acc: 0.55000
[Train] Step: 7500, loss: 0.40000, acc: 0.60000
[Train] Step: 8000, loss: 0.75000, acc: 0.25000
[Train] Step: 8500, loss: 0.35000, acc: 0.65000
[Train] Step: 9000, loss: 0.50000, acc: 0.50000
[Train] Step: 9500, loss: 0.45000, acc: 0.55000
[Train] Step: 10000, loss: 0.45000,

[Train] Step: 76000, loss: 0.40000, acc: 0.60000
[Train] Step: 76500, loss: 0.45000, acc: 0.55000
[Train] Step: 77000, loss: 0.50000, acc: 0.50000
[Train] Step: 77500, loss: 0.55000, acc: 0.45000
[Train] Step: 78000, loss: 0.50000, acc: 0.50000
[Train] Step: 78500, loss: 0.55000, acc: 0.45000
[Train] Step: 79000, loss: 0.35000, acc: 0.65000
[Train] Step: 79500, loss: 0.40000, acc: 0.60000
[Train] Step: 80000, loss: 0.60000, acc: 0.40000
(2000, 3072)
(2000,)
[Test ] Step: 80000, acc: 0.50050
[Train] Step: 80500, loss: 0.50000, acc: 0.50000
[Train] Step: 81000, loss: 0.50000, acc: 0.50000
[Train] Step: 81500, loss: 0.70000, acc: 0.30000
[Train] Step: 82000, loss: 0.35000, acc: 0.65000
[Train] Step: 82500, loss: 0.40000, acc: 0.60000
[Train] Step: 83000, loss: 0.60000, acc: 0.40000
[Train] Step: 83500, loss: 0.30000, acc: 0.70000
[Train] Step: 84000, loss: 0.65000, acc: 0.35000
[Train] Step: 84500, loss: 0.55000, acc: 0.45000
[Train] Step: 85000, loss: 0.70000, acc: 0.30000
(2000, 3072)
(